

Chargement des packages


In [1]:
import pandas as pd
import numpy as np

Pour faire le lien avec Google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import des données

In [3]:
movies_metadata = pd.read_csv('/content/drive/MyDrive/Data cinéma/movies_metadata.csv',low_memory=False,usecols=['title','id','imdb_id','release_date','popularity','vote_average','vote_count','overview'])
movies_metadata=movies_metadata[~movies_metadata['overview'].isna()]

ratings = pd.read_csv('/content/drive/MyDrive/Data cinéma/ratings.csv',usecols=['userId','movieId','rating'])

links = pd.read_csv('/content/drive/MyDrive/Data cinéma/links.csv')
def format_imdb(x):
    return ('tt{:07}'.format(x))
links['imdb_id'] = links['imdbId'].transform(format_imdb)

Filtrage sur les données

In [4]:
taux_filtre = 0.99

m  = movies_metadata['vote_count'].quantile(taux_filtre-0.05)
movies_metadata_filtre = movies_metadata.copy()[movies_metadata['vote_count'] >= m]
movies_metadata_filtre['popularity'] = movies_metadata_filtre['popularity'].astype(float)

info_movies = ratings.groupby('movieId').agg({'movieId': 'count'})
info_movies.rename(columns={'movieId': 'nb_note'},inplace=True)
info_movies['movieId'] = info_movies.index
m2  = info_movies['nb_note'].quantile(taux_filtre-0.05)
info_movies_filtre = info_movies.copy()[info_movies['nb_note'] >= m2]

movies_metadata_filtre=movies_metadata_filtre.merge(links[['movieId','imdb_id']],on='imdb_id')

info_movies_filtre = info_movies_filtre[info_movies_filtre['movieId'].isin(movies_metadata_filtre['movieId'])]
movies_metadata_filtre = movies_metadata_filtre[movies_metadata_filtre['movieId'].isin(info_movies_filtre['movieId'])]
ratings_filtre = ratings[ratings['movieId'].isin(info_movies_filtre['movieId'])]

info_user_total = ratings.groupby('userId').agg({'userId': 'count'})
info_user = ratings_filtre.groupby('userId').agg({'userId': 'count'})
info_user.rename(columns={'userId': 'nb_note'},inplace=True)
info_user['userId'] = info_user.index
m3  = info_user['nb_note'].quantile(taux_filtre)
info_user_filtre = info_user.copy()[info_user['nb_note'] >= m3]
ratings_filtre = ratings_filtre[ratings_filtre['userId'].isin(info_user_filtre['userId'])]

info_movies_filtre.reset_index(drop=True,inplace=True)
movies_metadata_filtre.reset_index(drop=True,inplace=True)
ratings_filtre.reset_index(drop=True,inplace=True)

print("En utilisant le filtre à",round(taux_filtre*100))
print("")
print("On garde",len(movies_metadata_filtre),"films ayant le plus de notes sur",len(movies_metadata),"soit",round(len(movies_metadata_filtre)/len(movies_metadata)*100),"%")
print("On conserve",len(info_user_filtre),"individus sur",len(info_user_total),"soit",round(len(info_user_filtre)/len(info_user_total)*100),"%")
print("On conserve",len(ratings_filtre),"notes sur",len(ratings),"soit",round(len(ratings_filtre)/len(ratings)*100),"%")

En utilisant le filtre à 99

On garde 1537 films ayant le plus de notes sur 44512 soit 3 %
On conserve 2689 individus sur 270896 soit 1 %
On conserve 1921422 notes sur 26024289 soit 7 %


Approche 1 Imputation

In [3]:
ratings_filtre = pd.read_csv('/content/drive/MyDrive/Data cinéma/ratings_filtre.csv')

In [88]:
# 36 min avec taux_filtre = 0.95
#  2 min avec taux_filtre = 0.99

from sklearn.impute import KNNImputer

rating_pivot = ratings_filtre.pivot(index='userId', columns='movieId', values='rating')
imputer = KNNImputer(n_neighbors=30, weights="uniform")
rating_pivot_imp = pd.DataFrame(imputer.fit_transform(rating_pivot))
rating_pivot_imp.columns = rating_pivot.columns
rating_pivot_imp['userId'] = rating_pivot.index
rating_imp = rating_pivot_imp.melt(id_vars=['userId'], var_name='movieId', value_name='rating_imp')

rating_imp.to_csv('/content/drive/MyDrive/Data cinéma/rating_imp.csv',index=False)  

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

rating_pivot = ratings_filtre.pivot(index='userId', columns='movieId', values='rating')
imputer = IterativeImputer(random_state=42,estimator=ExtraTreesRegressor(n_estimators=10, random_state=0))
imputed = imputer.fit_transform(rating_pivot)
df_imputed = pd.DataFrame(imputed, columns=rating_pivot.columns)
df_imputed['userId'] = rating_pivot.index
rating_imp_Iterat_ETR = df_imputed.melt(id_vars=['userId'], var_name='movieId', value_name='rating_imp_Iterat_ETR')

rating_imp_Iterat_ETR.to_csv('/content/drive/MyDrive/Data cinéma/rating_imp_Iterat_ETR.csv',index=False) 


In [40]:
movies_metadata_filtre.to_csv('/content/drive/MyDrive/Data cinéma/movies_metadata_filtre.csv',index=False)   

In [5]:
rating_imp = pd.read_csv('/content/drive/MyDrive/Data cinéma/rating_imp.csv')


In [6]:
rating_imp.shape
# debut 13h20

(16691808, 3)

In [7]:
#from sklearn.decomposition import PCA
#rating_pivot_imp = rating_imp.pivot(index='movieId', columns='userId', values='rating_imp')
#rating_pivot_imp.head()

#acp = PCA(svd_solver='full')
#coord_ACP = acp.fit_transform(rating_pivot_imp)
#print(acp.n_components_)
#print(acp.explained_variance_)

Approche 2 Text minning

In [47]:
import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

tokenizer = nltk.RegexpTokenizer(r'\w+')
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()
tfidf = TfidfVectorizer(stop_words='english')

def tokenize_mot_vide_lemm(a):
  # Etape 1 : tokenize
  Sac_de_mots=tokenizer.tokenize(a)
  # Etape 2 : suppression des mots vides et Etape 3 : lemmatizer
  F2 = [lemmatizer.lemmatize(x) for x in Sac_de_mots if x not in stopwords]
  return (" ".join(F2))

base_toke_vide_lemm = [tokenize_mot_vide_lemm(x) for x in movies_metadata_filtre['overview']]
tfidf_matrix = tfidf.fit_transform(base_toke_vide_lemm)

from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(movies_metadata_filtre.index, index=movies_metadata_filtre['title'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
cosine_sim = pd.read_csv('/content/drive/MyDrive/Data cinéma/cosine_sim.csv')
cosine_sim = np.array(cosine_sim)
indices = pd.Series(movies_metadata_filtre.index, index=movies_metadata_filtre['title'])


(1557,)
(1557, 9)


In [48]:
pd.DataFrame(cosine_sim).to_csv('/content/drive/MyDrive/Data cinéma/cosine_sim.csv',index=False)   

In [50]:
def get_recommendations(title):
    idx = indices[title]
    df = cosine_sim[idx]
    if len(df) < len(indices):
      df = np.mean(df, axis=0)
    sim_scores = list(enumerate(df))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:(10+len(idx))]
    movie_indices = [i[0] for i in sim_scores if i[0] not in list(idx)]
    # Return the top 10 most similar movies
    return movies_metadata_filtre[['title','release_date','vote_average','vote_count']].iloc[movie_indices]

In [51]:
get_recommendations(['The Dark Knight Rises'])

,title,release_date,vote_average,vote_count
1217,The Dark Knight,2008-07-16,8.3,12269.0
30,Batman Forever,1995-06-16,5.2,1529.0
105,Batman,1989-06-23,7.0,2145.0
277,Batman Returns,1992-06-19,6.6,1706.0
1038,Batman Begins,2005-06-10,7.5,7511.0
298,Batman & Robin,1997-06-20,4.2,1447.0
598,JFK,1991-12-20,7.5,513.0
1540,Batman v Superman: Dawn of Justice,2016-03-23,5.7,7189.0
1312,Law Abiding Citizen,2009-10-15,7.2,1522.0
1507,American Sniper,2014-12-11,7.4,4600.0


In [66]:
info_film = movies_metadata_filtre.sort_values('popularity', ascending=False)[['movieId','title','release_date','vote_count', 'vote_average', 'popularity']]


In [79]:
def get_recommendations_text_minning(note_ind):
  title = note_ind.merge(info_film,on='movieId')['title']
  idx = indices[title]
  df = cosine_sim[idx]
  if len(df) < len(indices):
    df = np.mean(df, axis=0)
  sim_scores = list(enumerate(df))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[0:(10+len(idx))]
  movie_indices = [i[0] for i in sim_scores if i[0] not in list(idx)]
  resultat = movies_metadata_filtre[['title','release_date','vote_average','vote_count']].iloc[movie_indices]
  resultat['similarity'] = df[movie_indices]
  # Return the top 10 most similar movies
  return resultat

In [84]:
note_ind = rating_imp[['movieId','rating_imp','userId']].iloc[lambda df: [0, 10000], :]
get_recommendations_text_minning(note_ind)


,title,release_date,vote_average,vote_count,similarity
1336,Toy Story 3,2010-06-16,7.6,4710.0,0.255715
565,Toy Story 2,1999-10-30,7.3,3914.0,0.231712
1047,The 40 Year Old Virgin,2005-08-11,6.2,2020.0,0.153285
184,Rebel Without a Cause,1955-10-27,7.6,351.0,0.088254
544,Licence to Kill,1989-07-07,5.9,549.0,0.082126
879,Lara Croft Tomb Raider: The Cradle of Life,2003-07-21,5.5,1443.0,0.076025
572,Man on the Moon,1999-12-22,6.9,444.0,0.074532
952,You Only Live Twice,1967-06-12,6.5,541.0,0.074005
545,Live and Let Die,1973-07-05,6.4,540.0,0.065084
1069,Match Point,2005-10-26,7.2,1134.0,0.064964


In [87]:
title = note_ind.merge(info_film,on='movieId')['title']

idx = indices[title]
print(idx)
df = cosine_sim[idx]
if len(df) < len(indices):
  df = np.mean(df, axis=0)
sim_scores = list(enumerate(df))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[0:(10+len(idx))]
movie_indices = [i[0] for i in sim_scores if i[0] not in list(idx)]
  # Return the top 10 most similar movies
resultat = movies_metadata_filtre[['title','release_date','vote_average','vote_count']].iloc[movie_indices]
resultat['similarity'] = df[movie_indices]
resultat

title
Batman    105
dtype: int64


,title,release_date,vote_average,vote_count,similarity
1410,The Dark Knight Rises,2012-07-16,7.6,9263.0,0.271171
298,Batman & Robin,1997-06-20,4.2,1447.0,0.176259
1038,Batman Begins,2005-06-10,7.5,7511.0,0.174531
1217,The Dark Knight,2008-07-16,8.3,12269.0,0.170455
277,Batman Returns,1992-06-19,6.6,1706.0,0.162285
30,Batman Forever,1995-06-16,5.2,1529.0,0.136049
1540,Batman v Superman: Dawn of Justice,2016-03-23,5.7,7189.0,0.120275
1085,Brick,2005-01-30,6.8,366.0,0.116121
781,We Were Soldiers,2002-03-01,6.7,531.0,0.110428
1009,Ong-Bak: The Thai Warrior,2003-01-21,7.0,556.0,0.100559


In [11]:
rating_pivot_imp = rating_imp.pivot(index='userId', columns='movieId', values='rating_imp')

2460.09157